# Development of machine learning models to process Electronic Health Records – Explainable Models

### Preprocessing Notebook
Lok Hang Toby Lee (2431180L)

# Data Extraction
---------------------------------------------------

### Configuration Step

In [6]:
# Imports:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mc
import colorsys
import psycopg2
import os
import yaml
%matplotlib inline


#pg_ctl.exe restart -D "E:\PostgreSQL\data"

# Configuration:
sqluser = 'postgres'
dbname = 'mimic'
password='postgres'
schema_name = 'public, mimic, mimiciii;'

# Connect to MIMIC-III:
con = psycopg2.connect(dbname=dbname, user=sqluser, password=password)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)


# SET YOUR PATH FOR RESOURCES FILE HERE
resources_path = "E:/MIMIC-III-ML/Workspace/resources/"
data_path = "E:/MIMIC-III-ML/Workspace/data/"

### Study cohort selection
- Only first ICU admissions that took at least a day and less than 10 days
- Adult patients only (age >= 15)

In [7]:
# Settings for the query:
min_age = 15
limit_population = 0 # if we want to run the query for a small number of patients (for debugging)
if limit_population > 0:
    limit = 'LIMIT ' + str(limit_population)
else:
    limit = ''
    
query = """
with patient_and_icustay_details as (
    SELECT distinct
        p.gender, p.dob, p.dod, s.*, a.admittime, a.dischtime, a.deathtime, a.ethnicity, a.diagnosis,
        DENSE_RANK() OVER (PARTITION BY a.subject_id ORDER BY a.admittime) AS hospstay_seq,
        DENSE_RANK() OVER (PARTITION BY s.hadm_id ORDER BY s.intime) AS icustay_seq,
        DATE_PART('year', s.intime) - DATE_PART('year', p.dob) as admission_age,
        DATE_PART('day', s.outtime - s.intime) as los_icu
    FROM patients p 
        INNER JOIN icustays s ON p.subject_id = s.subject_id
        INNER JOIN admissions a ON s.hadm_id = a.hadm_id 
    WHERE s.first_careunit NOT like 'NICU'
        and s.hadm_id is not null and s.icustay_id is not null
        and (s.outtime >= (s.intime + interval '12 hours'))
        and (s.outtime <= (s.intime + interval '240 hours'))
    ORDER BY s.subject_id 
)
SELECT * 
FROM patient_and_icustay_details 
WHERE hospstay_seq = 1
    and icustay_seq = 1
    and admission_age >=  """ + str(min_age) + """
    and los_icu >= 0.5
""" + str(limit)
patients_data = pd.read_sql_query('SET search_path to ' + schema_name + query, con)

# Save result:
#patients_data.to_csv('static_data.csv')

In [8]:
variables_to_keep = ('Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen', 
                     'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total',
                     'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure',
                     'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH')
var_map = pd.read_csv(resources_path + 'itemid_to_variable_map.csv')

In [9]:
icu_ids_to_keep = patients_data['icustay_id']
icu_ids_to_keep = tuple(set([str(i) for i in icu_ids_to_keep]))
subjects_to_keep = patients_data['subject_id']
subjects_to_keep = tuple(set([str(i) for i in subjects_to_keep]))
hadms_to_keep = patients_data['hadm_id']
hadms_to_keep = tuple(set([str(i) for i in hadms_to_keep]))

labitems_to_keep = []
chartitems_to_keep = []
for i in range(var_map.shape[0]):
    if var_map['LEVEL2'][i] in variables_to_keep:
        if var_map['LINKSTO'][i] == 'chartevents':
            chartitems_to_keep.append(var_map['ITEMID'][i])
        elif var_map['LINKSTO'][i] == 'labevents':
            labitems_to_keep.append(var_map['ITEMID'][i])
            
all_to_keep = chartitems_to_keep + labitems_to_keep
var_map = var_map[var_map.ITEMID.isin(all_to_keep)]
chartitems_to_keep = tuple(set([str(i) for i in chartitems_to_keep]))
labitems_to_keep = tuple(set([str(i) for i in labitems_to_keep]))

In [10]:
query = """
with patient_and_icustay_details as (
    SELECT distinct
        p.gender, p.dob, p.dod, s.*, a.admittime, a.dischtime, a.deathtime, a.ethnicity, a.diagnosis,
        DENSE_RANK() OVER (PARTITION BY a.subject_id ORDER BY a.admittime) AS hospstay_seq,
        DENSE_RANK() OVER (PARTITION BY s.hadm_id ORDER BY s.intime) AS icustay_seq,
        DATE_PART('year', s.intime) - DATE_PART('year', p.dob) as admission_age,
        DATE_PART('day', s.outtime - s.intime) as los_icu
    FROM patients p 
        INNER JOIN icustays s ON p.subject_id = s.subject_id
        INNER JOIN admissions a ON s.hadm_id = a.hadm_id 
    WHERE s.first_careunit NOT like 'NICU'
        and s.hadm_id is not null and s.icustay_id is not null
        and (s.outtime >= (s.intime + interval '12 hours'))
        and (s.outtime <= (s.intime + interval '240 hours'))
    ORDER BY s.subject_id 
)
SELECT * 
FROM patient_and_icustay_details 
WHERE hospstay_seq = 1
    and icustay_seq = 1
    and admission_age >=  """ + str(min_age) + """
    and los_icu >= 0.5
""" + str(limit)
patients_data = pd.read_sql_query('SET search_path to ' + schema_name + query, con)

# Save result:
#patients_data.to_csv(data_path + 'static_data.csv')

In [11]:
query = """
SELECT c.subject_id, i.hadm_id, c.icustay_id, c.charttime, c.itemid, c.value, c.valueuom
FROM icustays i
INNER JOIN chartevents c ON i.icustay_id = c.icustay_id
where c.icustay_id in """ + str(icu_ids_to_keep) + """
  and c.itemid in """ + str(chartitems_to_keep) + """
  and c.charttime between intime and outtime
  and c.error is distinct from 1
  and c.valuenum is not null
UNION ALL
SELECT distinct i.subject_id, i.hadm_id, i.icustay_id, l.charttime, l.itemid, l.value, l.valueuom
FROM icustays i
INNER JOIN labevents l ON i.hadm_id = l.hadm_id
where i.icustay_id in """ + str(icu_ids_to_keep) + """
  and l.itemid in """ + str(labitems_to_keep) + """
  and l.charttime between (intime - interval '6' hour) and outtime
  and l.valuenum > 0 -- lab values cannot be 0 and cannot be negative
"""
events_data = pd.read_sql_query('SET search_path to ' + schema_name + query, con)
events_data.to_csv(data_path + 'events_data.csv')

In [12]:
itemids = tuple(set(events_data.itemid.astype(str)))
events_data.head()

,subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom
0,266,186251,293876,2168-07-11 14:00:00,220179,132,mmHg
1,266,186251,293876,2168-07-11 14:00:00,220180,78,mmHg
2,266,186251,293876,2168-07-11 14:00:00,220181,89,mmHg
3,266,186251,293876,2168-07-11 14:00:00,220210,17,insp/min
4,266,186251,293876,2168-07-11 15:00:00,220045,75,bpm


In [13]:
query_d_items = \
        """
        SELECT itemid, label, dbsource, linksto, category, unitname
        FROM d_items
        WHERE itemid in """ + str(itemids)
d_output = pd.read_sql_query('SET search_path to ' + schema_name + query_d_items, con)

In [14]:
# Remove the text from the categorical (Glasgow coma scale) variables so we can make them numeric:
replacement_dictionary = {'4 Spontaneously': '4', '3 To speech': '3', '2 To pain': '2', '1 No Response': '1',
                         '5 Oriented': '5', '1.0 ET/Trach': '1', '4 Confused': '4', '2 Incomp sounds': '2', 
                         '3 Inapprop words': '3', 'Spontaneously': '4', 'To Speech': '3', 'None': '1', 'To Pain': '2',
                         '6 Obeys Commands': '6', '5 Localizes Pain': '5', '4 Flex-withdraws': '4', '2 Abnorm extensn': '2',
                         '3 Abnorm flexion': '3', 'No Response-ETT': '1', 'Oriented': '5', 'Confused': '4', 
                         'No Response': '1', 'Incomprehensible sounds': '2', 'Inappropriate Words': '3', 
                         'Obeys Commands': '6', 'No response': '1', 'Localizes Pain': '5', 'Flex-withdraws': '4',
                         'Abnormal extension': '2', 'Abnormal flexion': '3', 'Abnormal Flexion': '3', 
                          'Abnormal Extension': '2'}
for key, value in replacement_dictionary.items():
    events_data['value'] = events_data['value'].replace(key, value) 

In [15]:
# Change data types and set indices:
events_data['value'] = pd.to_numeric(events_data['value']) #, 'coerce')
events_data = events_data.astype({k: int for k in ['subject_id', 'hadm_id', 'icustay_id']})
patients_data = patients_data.reset_index().set_index('icustay_id')
var_map = var_map[['LEVEL2', 'ITEMID', 'LEVEL1']].rename(
    {'LEVEL2': 'LEVEL2', 'LEVEL1': 'LEVEL1', 'ITEMID': 'itemid'}, axis=1).set_index('itemid')

# Change to hourly data:
to_hours = lambda x: max(0, x.days*24 + x.seconds // 3600)
events_data = events_data.set_index('icustay_id').join(patients_data[['intime']])
events_data['hours_in'] = (events_data['charttime'] - events_data['intime']).apply(to_hours)
events_data = events_data.drop(columns=['charttime', 'intime']) 

# Join with d_output query and group variables:
events_data = events_data.set_index('itemid', append=True)
events_data = events_data.join(var_map)
d_output = d_output.set_index('itemid')
events_data = events_data.join(d_output) 
events_data = events_data.set_index(['label', 'LEVEL1', 'LEVEL2'], append=True)
patients_data['max_hours'] = (patients_data['outtime'] - patients_data['intime']).apply(to_hours)

In [16]:
# Save results:
patients_data.to_hdf(data_path + 'vitals_hourly_data.h5', 'patients_data')
events_data.to_hdf(data_path + 'vitals_hourly_data.h5', 'X')

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['gender', 'dbsource', 'first_careunit', 'last_careunit', 'ethnicity', 'diagnosis']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['valueuom', 'dbsource', 'linksto', 'category', 'unitname']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


### Extract length of stay and in-hospital mortality

In [17]:
outcomes = pd.DataFrame(index=patients_data.index)
# In hospital mortality: patient has died after the admittime to hospital and before the outtime:
mortality = patients_data.dod.notnull() & ((patients_data.admittime <= patients_data.dod) & (patients_data.outtime >= patients_data.dod))
mortality = mortality | (patients_data.deathtime.notnull() & ((patients_data.admittime <= patients_data.deathtime) & 
                                                             (patients_data.dischtime >= patients_data.deathtime)))
outcomes['in_hospital_mortality'] = mortality.astype(int)

# Length of stay (in hours):
outcomes['los'] = patients_data['los'] * 24.0
outcomes.to_hdf(data_path + 'vitals_hourly_data.h5', 'Y')

In [18]:
#Save to csv
events_data.to_csv(data_path + 'events_data.csv')
patients_data.to_csv(data_path + 'patients_data.csv')
outcomes.to_csv(data_path + 'outcomes.csv')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Preprocessing
----------------------------------------------------------

In [19]:
# Imports:
import numpy as np
import pandas as pd
import os
import json

In [20]:
events_data = pd.read_hdf(data_path + 'vitals_hourly_data.h5', 'X')
events_data = events_data.reset_index()
print('events_data (X): ', events_data.shape)

patients_data = pd.read_hdf(data_path + 'vitals_hourly_data.h5', 'patients_data')
print('patients_data: ', patients_data.shape)

outcomes = pd.read_hdf(data_path + 'vitals_hourly_data.h5', 'Y')
print('outcomes (Y): ', outcomes.shape)

# Load the config file that contains information about continuous/categorical variables:
config = json.load(open(resources_path + 'discretizer_config.json', 'r'))
is_categorical = config['is_categorical_channel']

# Get categorical variables:
categorical_var = []
continuous_var = []
for key, value in is_categorical.items():
    if value:
        categorical_var.append(key)
    else:
        continuous_var.append(key)
print('Categorical: ', categorical_var[1:])
print('Continuous: ', continuous_var)
categorical_var = categorical_var[1:]

events_data (X):  (20567026, 14)
patients_data:  (30063, 25)
outcomes (Y):  (30063, 2)
Categorical:  ['Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total', 'Glascow coma scale verbal response']
Continuous:  ['Diastolic blood pressure', 'Fraction inspired oxygen', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure', 'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH']


### Pre-processing steps

In [21]:
# Map variables to the same metric:
UNIT_CONVERSIONS = [
    ('weight',                   'oz',  None,             lambda x: x/16.*0.45359237),
    ('weight',                   'lbs', None,             lambda x: x*0.45359237),
    ('fraction inspired oxygen', None,  lambda x: x > 1,  lambda x: x/100.),
    ('oxygen saturation',        None,  lambda x: x <= 1, lambda x: x*100.),
    ('temperature',              'f',   lambda x: x > 79, lambda x: (x - 32) * 5./9),
    ('height',                   'in',  None,             lambda x: x*2.54),
]

variable_names = events_data['LEVEL1'].str
variable_units = events_data['valueuom'].str
for name, unit, check, convert_function in UNIT_CONVERSIONS:
    indices_variable = variable_names.contains(name, case=False, na=False)
    needs_conversion_filter_indices = indices_variable & False
    if unit is not None:
        needs_conversion_filter_indices |= variable_names.contains(unit, case=False, na=False) | variable_units.contains(unit, case=False, na=False)
    if check is not None:
        needs_conversion_filter_indices |= check(events_data['value'])
    idx = indices_variable & needs_conversion_filter_indices
    events_data.loc[idx, 'value'] = convert_function(events_data['value'][idx])

In [23]:
# Detect and remove outliers. For this, they use two different outlier ranges: 
# 1) for each variable, they have an upper and lower threshold for detecting unusable outliers. 
#    If the outlier falls outside of these threshold, it is treated as missing. 
# 2) they also have a physiologically valid range of measurements. If the non-outlier falls outside this range, 
     # it is replaced with the nearest valid value.

variable_ranges = pd.read_csv(resources_path + 'variable_ranges.csv', index_col = None)
variable_ranges['LEVEL2'] = variable_ranges['LEVEL2'].str.lower()
variable_ranges = variable_ranges.set_index('LEVEL2')

variables_all = events_data['LEVEL2']
non_null_variables = ~events_data.value.isnull()
variables = set(variables_all)
range_names = set(variable_ranges.index.values)
range_names = [i.lower() for i in range_names]

for var_name in variables:
    var_name_lower = var_name.lower()
    
    if var_name_lower in range_names:
        out_low, out_high, val_low, val_high = [
            variable_ranges.loc[var_name_lower, x] for x in ('OUTLIER LOW', 'OUTLIER HIGH', 'VALID LOW', 'VALID HIGH')
        ]
        
        # First find the indices of the variables that we need to check for outliers:
        indices_variable = non_null_variables & (variables_all == var_name)
        
        # Check for low outliers and if they are not extreme, replace them with the imputation value:
        outlier_low_indices = (events_data.value < out_low)
        low_not_outliers = ~outlier_low_indices & (events_data.value < val_low)
        valid_low_indices = indices_variable & low_not_outliers
        events_data.loc[valid_low_indices, 'value'] = val_low
        
        # Check for high outliers and if they are not extreme, replace them with the imputation value:
        outlier_high_indices = (events_data.value > out_high)
        high_not_outliers = ~outlier_high_indices & (events_data.value > val_high)
        valid_high_indices = indices_variable & high_not_outliers
        events_data.loc[valid_high_indices, 'value'] = val_high
        
        # Treat values that are outside the outlier boundaries as missing:
        outlier_indices = indices_variable & (outlier_low_indices | outlier_high_indices)
        events_data.loc[outlier_indices, 'value'] = np.nan

### Reshape data
We want to have a column for every variable:

In [24]:
events_data = events_data.set_index(['icustay_id', 'itemid', 'label', 'LEVEL1', 'LEVEL2'])
events_data = events_data.groupby(['icustay_id', 'subject_id', 'hadm_id', 'LEVEL2', 'hours_in'])
events_data = events_data.agg(['mean', 'std', 'count'])
events_data.columns = events_data.columns.droplevel(0)
events_data.columns.names = ['Aggregation Function']
events_data = events_data.unstack(level = 'LEVEL2')
events_data.columns = events_data.columns.reorder_levels(order=['LEVEL2', 'Aggregation Function'])

In [25]:
# Make sure we have a row for every hour:
missing_hours_fill = pd.DataFrame([[i, x] for i, y in patients_data['max_hours'].iteritems() for x in range(y+1)],
                                 columns=[patients_data.index.names[0], 'hours_in'])
missing_hours_fill['tmp'] = np.NaN

fill_df = patients_data.reset_index()[['subject_id', 'hadm_id', 'icustay_id']].join(
     missing_hours_fill.set_index('icustay_id'), on='icustay_id')
fill_df.set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in'], inplace=True)

events_data = events_data.reindex(fill_df.index)
events_data = events_data.sort_index(axis = 0).sort_index(axis = 1)

idx = pd.IndexSlice
events_data.loc[:, idx[:, 'count']] = events_data.loc[:, idx[:, 'count']].fillna(0)

In [26]:
# Save this version of the data as a .csv file, so we can apply different imputation methods in another notebook:
idx = pd.IndexSlice
timeseries_data = events_data.loc[:, idx[:, 'mean']]
timeseries_data = timeseries_data.droplevel('Aggregation Function', axis = 1) 
timeseries_data = timeseries_data.reset_index() 
timeseries_data.to_csv(data_path + 'mimic_timeseries_data_not_imputed.csv')

### Imputation of time series data

In [27]:
idx = pd.IndexSlice
timeseries_data = events_data.loc[:, idx[:, ['mean', 'count']]]

# Get the mean across hours for each variable and each patient:
icustay_means = timeseries_data.loc[:, idx[:, 'mean']].groupby(['subject_id', 'hadm_id', 'icustay_id']).mean()

# Get the global mean for each variable:
global_means = timeseries_data.loc[:, idx[:, 'mean']].mean(axis = 0)

# Forward fill the nan time series, or otherwise fill in the patient's mean or global mean:
timeseries_data.loc[:, idx[:, 'mean']] = timeseries_data.loc[:, idx[:, 'mean']].groupby(
    ['subject_id', 'hadm_id', 'icustay_id']).fillna(method='ffill').groupby(
    ['subject_id', 'hadm_id', 'icustay_id']).fillna(icustay_means).fillna(global_means)

# Create a mask that indicates if the variable is present:
timeseries_data.loc[:, idx[:, 'count']] = (events_data.loc[:, idx[:, 'count']] > 0).astype(float)
timeseries_data.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)

# Add a variable that indicates the time since the last measurement to the dataframe:
is_absent = (1 - timeseries_data.loc[:, idx[:, 'mask']])
hours_of_absence = is_absent.cumsum()
time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)
timeseries_data = pd.concat((timeseries_data, time_since_measured), axis = 1)
timeseries_data.loc[:, idx[:, 'time_since_measured']] = timeseries_data.loc[:, idx[:, 'time_since_measured']].fillna(100)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


### Standardization of continuous data

In [28]:
# Minmax standardization:
def minmax(x):
    mins = x.min()
    maxes = x.max()
    x_std = (x - mins) / (maxes - mins)
    return x_std

def std_time_since_measurement(x):
    idx = pd.IndexSlice
    x = np.where(x==100, 0, x)
    means = x.mean()
    stds = x.std() + 0.0001
    x_std = (x - means)/stds
    return x_std

timeseries_data.loc[:, idx[continuous_var, 'mean']] = timeseries_data.loc[:, idx[continuous_var, 'mean']].apply(lambda x: minmax(x))
timeseries_data.loc[:, idx[:, 'time_since_measured']] = timeseries_data.loc[:, idx[:, 'time_since_measured']].apply(lambda x: std_time_since_measurement(x))

### One-hot encoding categorical variables

In [29]:
# First we need to round the categorical variables to the nearest category:
categorical_data = timeseries_data.loc[:, idx[categorical_var, 'mean']].copy(deep=True)
categorical_data = categorical_data.round()
one_hot = pd.get_dummies(categorical_data, columns=categorical_var)

# Clean up the columns that we do not need and add the dummy encodings:
for c in categorical_var:
    if c in timeseries_data.columns:
        timeseries_data.drop(c, axis = 1, inplace=True)
timeseries_data.columns = timeseries_data.columns.droplevel(-1)
timeseries_data = pd.merge(timeseries_data.reset_index(), one_hot.reset_index(), how='inner', left_on=['subject_id', 'icustay_id', 'hadm_id', 'hours_in'],
                           right_on=['subject_id', 'icustay_id', 'hadm_id', 'hours_in'])
timeseries_data = timeseries_data.set_index(['subject_id', 'icustay_id', 'hadm_id', 'hours_in'])

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


### Preprocessing of Y / outcomes

In [30]:
# First get the number of nan values per variable:
print(outcomes.isna().sum())

# We will replace them with zero:
outcomes = outcomes.fillna(0)

in_hospital_mortality    0
los                      0
dtype: int64


### Save all pre-processed data

In [31]:
# Rename the columns and save the results:
s = timeseries_data.columns.to_series()
timeseries_data.columns = s + s.groupby(s).cumcount().astype(str).replace({'0':''})

timeseries_data.to_hdf(data_path + 'vitals_hourly_data_preprocessed.h5', 'X')
outcomes.to_hdf(data_path + 'vitals_hourly_data_preprocessed.h5', 'Y')